# WEEK1    
## **数据集**
1. 训练集(train set)
2. 简单交叉验证集/验证集(hold-out cross validation set/dev set) 
3. 测试集(test set) 无偏评估

数据集较小情况:70%(train+dev)/30%(test)分割或者60%(train)/20%(dev)/20%(test)

- **保证验证集与数据集来自同一分布**   
假设一个分辨猫的照片的神经网络，可能会出现数据库中是抓取的精修照片，用户上传的是随意拍摄的，即不同分布     


- 使用验证集是为了**快速调参**，就是**用验证集选择超参数**。另外用验证集还可以监控模型是否异常，决定是不是要提前停止训练。       

- 测试集既**不参与参数的学习过程，也不参与参数的选择过程**，仅仅用于模型评价。      

## 偏差与方差       
偏差与方差的权衡是评测模型优度的一个标准     
首先方差与偏差是基于**假设预测**的，理想的误差被称为**最优误差/基本误差（Base error）**，例如分别猫与狗中，人的分类误差可以看作Base error   

|  训练集误差   | 验证集误差  |结果  |
|  :-:  | :-: |:-: |
| (相对于Base error)低  | (相对于Base error)低 |低偏差,低误差,优良|
| 低  | 高 | 高方差 | 
| 高  | 与Train error基本持平 | 高偏差 |
| 高  | 高于Train error | 高偏差 高方差 | 

- 红线偏差与方差较低
- 黄线由于几乎是线性拟合器，但又在中间部分较为灵活，是高偏差与高方差的
- 紫线是线性分类器，因此拟合不充足，是高偏差
<img style="float: center;" src="course_2_pics/week_1_1.jpg" width=500 height=500>



## 模型优化     
<img style="float: center;" src="course_2_pics/week_1_2.png" width=500 height=500>   

## 正则化      
$$J(w^{[1]},b^{[1]},...,w^{[L]},b^{[L]})=\frac{1}{m}\sum^{n}_{i=1}L(\hat y^{(i)},y^{(i)})+\frac{\lambda}{2m}\sum^{L}_{l=1}||w^{[l]}||$$    
- **L2范数**:$||w||^{2}_{2}=\sum^{n_x}_{j=1}w_j^2=w^Tw$    


- **L1范数**:$\sum^{n_x}_{j=1} |w_j|=||w||_1$    

$\lambda$被称为正则化参数，用验证集配置(lambda是python中的一个预留字段,因此使用lambd)
### 神经网络中的正则化

#### 优缺点

- L2正则化优化方便，可以直接求导； L2 一定只有一条最好的预测线(L1求导是双向逼近)；L2求导后比较均匀
- L1可以计算绝对值之和，用以产生稀疏性(对大规模的特征有意义)

原因是L2求导是$2\times w$,L1是$Constant=1$

`https://www.zhihu.com/question/26485586`

$$J(w^{[1]},b^{[1]},...,w^{[L]},b^{[L]})=\frac{1}{m}\sum^{n}_{i=1}L(\hat y^{(i)},y^{(i)})+\frac{\lambda}{2m}\sum^{L}_{l=1}||w^{[l]}||_F$$

$$||w^{[l]}||_F=\sum^{n^{[l-1]}}_i\sum^{n^{[l]}}_j(w_{ij}^{[l]})^2$$

λ的值是可以使用开发集调整时的超参数。L2正则化会**使决策边界更加平滑**。如果λ太大，也可能会“过度平滑”，从而导致模型高偏差。        
L2正则化依赖于**较小权重的模型比具有较大权重的模型更简单**这样的假设。        
因此，通过削减成本函数中权重的平方值，可以将所有权重值逐渐改变到到较小的值。            
λ数值高的话会有更平滑的模型，其中**输入变化时输出变化更慢**，但是你需要花费更多的时间。
 
#### L2正则化对以下内容有影响：
1. 成本计算：正则化的计算需要添加到成本函数中
2. 反向传播功能：在权重矩阵方面，梯度计算时也要依据正则化来做出相应的计算
3. 重量变小（“重量衰减”)：权重被逐渐改变到较小的值。


- **反向传播过程:**
$$\frac{\partial J}{\partial w^{[l]}}=dw^{[l]}=(backprob)+\frac{\lambda}{m}w^{[l]}$$
$$w^{[l]}=w^{[l]}-\alpha((backprob)+\frac{\lambda}{m}w^{[l]})=w^{[l]}(1-\alpha \frac{\lambda}{m})-\alpha(backprob)$$

**相当于在原本的$w^{[l]}$乘以$(1-\alpha \frac{\lambda}{m})$再进行反向传播**       

(Frobenius norm 弗洛贝尼乌斯范数，下标F)   

- **正则化的作用过程**    

对于一个神经网络而言，理论上在正则化参数$\lambda$足够大时，多个单元的$w\rightarrow0$，因此会神经网络会简化为接近一个深度较大的逻辑回归，从过拟合的状态转为线性分类器状态。$\lambda$处于中间值时，分类器会有较小的偏差和方差。
<img style="float: center;" src="course_2_pics/week_1_3.jpg" width=500 height=500>  


对于激活函数$tanh(z)$而言，$z^{[l]}=w^{[l]}a^{[l-1]}+b^{[l]}$，那么当$w^{[l]}$越小，激活函数$tanh(z)$在取值越小时越接近于线性函数，因此不会有过于复杂的非线性边界。
<img style="float: center;" src="course_2_pics/week_1_4.png" width=300 height=300> 

## DropOut正则化   
为神经网络的每一层设置一个概率，通过随机数决定每层的节点是否失活，失活节点删除进出连线，得到一个简易的神经网络     
<img style="float: center;" src="course_2_pics/week_1_5.png" width=5300 height=500> 

- **反向随机失活(invert dropout)**    
1. 假设一个三层神经网络$l=3$，不失活概率$keep\text{_}prob=0.2$     
2. 设置一个输入值矩阵$a3$与失活矩阵$d3=np.random.rand(a3.shape[0],a3.shape[1])<keep\text{_}prob$   
3. 更新$a3=a3\cdot d3$，并且将$a3/keep\text{_}prob$，为了不影响神经元的输出值期望，因此需要除以神经元保留概率   

- **Dropout的作用**   
可以防止单独地把权重依赖在某个输入特征上，从而达到让每个特征具有较为平等的重要性的作用       
在dropout时，缺点是损失函数的定义会变化   

## 其他正则化方法  
1. 数据扩增   
计算机视觉问题中，可以对图片进行翻转、扭曲、裁剪   
2. early stopping    
在训练过程中绘制训练误差与验证误差时，有时会发现训练误差下降但验证误差先下降后增高，可以选择**在表现较好的迭代数时停止训练**    

## 归一化    
$\mu,\sigma^2$通过训练集计算    

## 梯度消失与梯度爆炸                         

```https://blog.csdn.net/junjun150013652/article/details/81274958```
    
假设一个深度神经网络，每个有两个神经元，那么有    
$$\hat y = w^{[l]}w^{[l-1]}...w^{[2]}w^{[1]}x$$   
在这种状况下，如果$w^{[l]}=w^{[l-1]}=...w^{[1]}$，并且$w^{[l]}\gt 1$或$w^{[l]}\lt 1$，在深度较大的情况下，会导致$\hat y\rightarrow \infty$或$\hat y\rightarrow 0$，因此需要随机指定权重    
同样，在深度较大的情况下，权重大于1或小于1的叠加会导致梯度函数与激活函数的指数级增长或减小     


## 神经网络权重初始化         
<font color='red'> 推导权重反向传播过程 </font>     
在权重初始化过程中，$z = w_1x_1+w_2x_2+....w_nx_n$，当期望$z$小一些，则希望$w$小一些，因此设置$Var(W_i)=\frac{1}{n}$     

**基于方差缩放的:需要尽量保持每个神经元输入和输出方差一致**                  

在python中实现:    
Sigmoid:$ np.random.randn(shape)*np.sqrt(\frac{1}{n^{[l-1]}}) $    
Relu:$np.sqrt(\frac{2}{n^{[l-1]}})$    
tanh$np.sqrt(\frac{1}{n^{[l-1]}})$  

使用Xavier:$\cfrac{1}{n^{[l-1]}  *n^{[l]}},n^{[l]}代表第l层的神经元个数代表$       

1. 权重为0:无法打破对称性，结果不变(gd无法起效)
2. 较大的随机权重，**最后一个激活(sigmoid)输出的结果非常接近于0或1**，而当它出现错误时，它会导致非常高的损失。初始化参数如果没有很好地话会导致梯度消失、爆炸，这也会减慢优化算法。如果对这个网络进行更长时间的训练，将看到更好的结果，但是使用**过大的随机数初始化会减慢优化的速度**。

## 梯度逼近     
- **双边误差(更精确的逼近)**  
图中是$f(\theta) = \theta ^3$的函数，观察两个小三角形构成的大三角形对梯度进行预估     
即$\cfrac{f(\theta +\varepsilon)-f(\theta -\varepsilon)}{2\varepsilon}$，即大三角形的高除以底，这样有$f'(\theta)=lim_{\varepsilon\rightarrow 0}\cfrac{f(\theta +\varepsilon)-f(\theta -\varepsilon)}{2\varepsilon}+O(\varepsilon^2)$
<img style="float: center;" src="course_2_pics/week_1_6.png" width=300 height=300> 
- **单边误差**   
利用单个小三角形对梯度进行预估     
即$\cfrac{f(\theta +\varepsilon)-f(\theta )}{\varepsilon}$，即大三角形的高除以底，这样有$f'(\theta)=lim_{\varepsilon\rightarrow 0}\cfrac{f(\theta +\varepsilon)-f(\theta )}{\varepsilon}+O(\varepsilon)$
<img style="float: center;" src="course_2_pics/week_1_7.png" width=300 height=300>     

## 梯度检验       

设一个$L$层的神经网络，将不同层的参数$W^{[1]},b^{[1]}...,W^{[L]},b^{[L]}$连接为一个参数矩阵$\theta$，得到一个成本函数$J(\theta)$     

将不同层参数的导数$dW^{[1]},db^{[1]}...,dW^{[L]},db^{[L]}$连接为一个参数矩阵$d\theta$ ,与$\theta$有相同的维度     

- **Grad check(梯度检验)**     
$J(\theta)=J(\theta_1,\theta_2,...\theta_L)$，由此可以得到：
1. 对每个$i$有:
$$d\theta_{approx}^{[i]}=\cfrac{J(\theta_1,\theta_2,\theta_i+\varepsilon...)-J(\theta_1,\theta_2,\theta_i-\varepsilon...)}{2\varepsilon}\approx d\theta^{[i]}=\cfrac{\partial J}{\partial \theta}$$      
2. 得到$$d\theta_{approx}\approx d\theta$$
3. 检查:
$$\cfrac{||d\theta_{approx}- d\theta||_2}{||d\theta_{approx}||_2+||d\theta||_2}$$     
假设取$\varepsilon=10^{-7}$，那么假设上述值非常小(假设小于$10^{-7}$)，那么认为梯度检验通过；如果较小，可以逐项检查是否有某项误差特别大；如果较大，那么检查全部项，可能出现问题。      

- 梯度检验的注意事项    
1. 梯度检验用于调试，不用于训练时使用
2. 尝试检查每一列，在$d\theta_{approx}与d\theta$差值特别大的某列确认错误   
3. 梯度检验时，成本函数是包括正则项的    
4. 不能与Dropout同时使用  
5. 在随机初始化时使用梯度检验，反复训练一段时间后再进行梯度检验。在训练时$w,b$可能会在值较小时梯度下降时正确的，当值变大后出现问题。